In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.api_wrappers import get_industry_employment_data

df_ilo = get_industry_employment_data(start_year=2020, end_year=2024, output_filename="ilo_data_raw.csv")
df_ilo.info()

Fetching ILO data from: https://rplumber.ilo.org/data/indicator/?id=EMP_TEMP_ECO_OCU_NB_A&timefrom=2020&timeto=2024&classif1=ECO_SECTOR_IND&classif2=OCU_SKILL_TOTAL&format=.csv
Saving the dataset to: ilo_data_raw.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427 entries, 0 to 426
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ref_area        427 non-null    object 
 1   source          427 non-null    object 
 2   indicator       427 non-null    object 
 3   classif1        427 non-null    object 
 4   classif2        427 non-null    object 
 5   time            427 non-null    int64  
 6   obs_value       427 non-null    float64
 7   obs_status      33 non-null     object 
 8   note_classif    0 non-null      float64
 9   note_indicator  34 non-null     object 
 10  note_source     427 non-null    object 
dtypes: float64(2), int64(1), object(8)
memory usage: 36.8+ KB
